# Collect articles about hurricane Helene coverage

1. Use wayback Machine to scrape everything from a news website

2. Use UNC access's News Bank

## 2. News Bank pdf about "Helene" coverage

Use Selenium to download news article pdfs from news bank

keyword: helene

date: Sep 2024 (when the first 3 articles about Helene were published) - Oct 2024?

Location: USA - North Carolina

In [ ]:
# Import Module
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Scrape multiple pages

In [ ]:
# Open Chrome
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

# Open URL
url = "https://infoweb.newsbank.com/apps/news/results?hide_duplicates=2&offset=0&maxresults=60&sort=YMD_date%3AD&p=AMNEWS&t=country%3AUSA%21USA/state%3ANC%21USA%2B-%2BNorth%2BCarolina&f=advanced&val-base-0=helene&fld-base-0=alltext&bln-base-1=and&val-base-1=Sep%202024%20-%20Oct%202024&fld-base-1=YMD_date"
driver.get(url)
time.sleep(5)
# Download pdf
try:
    # For I pages
    for i in range(2):
        # Click "select all" box
        # await driver.find_element(By.CLASS_NAME, 'search-hits__select-all form-checkbox').click()
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "search-hits__select-all.form-checkbox"))).click()
        # Click next
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "pager__item.pager__item--next"))).click()
        time.sleep(1)
except Exception as e:
    print(f"Cannot select page {i}")
    print(e)
else:
    # Click download button
    driver.find_element(By.CLASS_NAME, 'actions-bar__button.actions-bar__button--download').click()
    time.sleep(1)
    # While there is "Next" button
    while True:
        # Click "Download" button in pop-up
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "u-button multidoc-download__button"))).click()
        # wait loader to appear and then disappear
        WebDriverWait(driver, 10).until(EC.element_to_be_selected((By.CLASS_NAME, 'multidoc__button-spinner')))
        WebDriverWait(driver, 10).until(EC.invisibility_of_element((By.CLASS_NAME, 'multidoc__button-spinner')))
        # Click "Next"
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "multidocs_pager_nav next"))).click()
        except:
            print("No Next found")
            break

driver.quit()





There is security check of robot. Can't pass ?

Scrape day by day?

In [ ]:
import pdfplumber
import os
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By 
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re


extract text from pdf with <a href = 'https://github.com/jsvine/pdfplumber'>pdfplumber</a>

In [ ]:
#   FUNC extract_text_from_pdf()
#   Input: directory (str) of pdf
#   Output: header (list) - article header information
#         article (list) - article text
def extract_text_from_pdf(dir, headers, articles):
    # header, article = [],[]
    with pdfplumber.open(dir) as pdf:
        for page in pdf.pages:
            # extract texts on each page and split texts by "OpenURL Link"
            texts = page.extract_text().split("\nOpenURL Link\n")
            # there is no "OpenURL Link" on that page
            if len(texts) == 1:
                articles[-1] += texts[0]
            else:
                headers.append(texts[0])
                articles.append(texts[1])
    return 

In [ ]:
# FUNC: decompose_header
# Input: header (str)
# Output: append titles, dates, newspapers, authors, and word_counts
def decompose_header(header):


    date_match = re.search(r"\b(September|October) \d{1,2}, \d{4}\b", header)
    
    date = date_match.group(0) if date_match else ""
    dates.append(date)
    loc = header.find(date)
    titles.append(header[:loc])

    texts = header[loc:].split('\n')
    newspapers.append(texts[0][len(date)+1:])

    # Line three
    match = re.search(r"Author: (.*?)Section: ", texts[1])
    author = match.group(1) if match else ""
    authors.append(author)
    match_word = re.search(r"(\d+)\s*Words", texts[1])
    word_count = match_word.group(1) if match_word else ""
    word_counts.append(word_count)

    return

In [ ]:
# Get all files under news_bank_pdf
directory = 'news_bank_pdf'
pdf_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".pdf")]

headers, articles = [],[]
# Extract articles and headers from pdfs
for path in pdf_paths:
    extract_text_from_pdf(pdf_paths[0], headers, articles)

# Save articles and headers to dataframe
temp = pd.DataFrame({"header":headers,
                    "article": articles})

# temp.to_csv('articles_20241107.csv', index=False)

In [ ]:
# Decompose headers
titles, dates, newspapers, authors, word_counts = [],[],[],[],[]
temp['header'].apply(lambda x: decompose_header(x))

temp['title'] = titles
temp['date'] = dates
temp['newspaper'] = newspapers
temp['author'] = authors
temp['word_count'] = word_counts
temp.head()

# temp.to_csv("articles_20241108.csv", index=False)

In [ ]:
def extract_text_tru_link(url):
    # Open Chrome
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)

    # Open URL
    # url = "https://infoweb.newsbank.com/apps/news/results?hide_duplicates=2&offset=0&maxresults=60&sort=YMD_date%3AD&p=AMNEWS&t=country%3AUSA%21USA/state%3ANC%21USA%2B-%2BNorth%2BCarolina&f=advanced&val-base-0=helene&fld-base-0=alltext&bln-base-1=and&val-base-1=Sep%202024%20-%20Oct%202024&fld-base-1=YMD_date"
    driver.get(url)
    time.sleep(5)

    # WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "search-hits__select-all.form-checkbox"))).click()
    driver.quit()
    return

extract_text_tru_link('https://infoweb.newsbank.com/apps/news/user/login?destination=results%3Fhide_duplicates%3D2%26offset%3D0%26maxresults%3D60%26sort%3DYMD_date%253AD%26p%3DAMNEWS%26t%3Dcountry%253AUSA%2521USA/state%253ANC%2521USA%252B-%252BNorth%252BCarolina%26f%3Dadvanced%26val-base-0%3Dhelene%26fld-base-0%3Dalltext%26bln-base-1%3Dand%26val-base-1%3DSep%25202024%2520-%2520Oct%25202024%26fld-base-1%3DYMD_date')


    